<a href="https://colab.research.google.com/github/deisyrojas2/Intro-IA/blob/main/03%20-%20modelo%20con%20Random%20Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-learn

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score , confusion_matrix , ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split , KFold
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score , confusion_matrix , ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split , KFold
from xgboost import XGBClassifier
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd '/content/drive/MyDrive/udea-IA-20242'

/content/drive/MyDrive/udea-IA-20242


In [ ]:
def convertir_si_no(valor):
    if isinstance(valor, str):  # Verifica si el valor es una cadena
        if valor.lower() == 'si' or valor.lower() == 's':
            return 1
        elif valor.lower() == 'no' or valor.lower() == 'n':
            return 0
    return valor  # Devuelve el valor original si no es 'si' ni 'no' o si no es una cadena

In [ ]:
def preproccesing(dir, test, size=0):

  data = pd.read_csv(dir)
  print(data.shape)
  columnas_string = data.select_dtypes(include='object').columns
  data[columnas_string] = data[columnas_string].fillna('0')
  label_encoders = {}
  for column in columnas_string:
    label_encoders[column] = LabelEncoder()
    data[column] = label_encoders[column].fit_transform(data[column])
  columnas_float = data.select_dtypes(include='float64').columns
  data[columnas_float] = data[columnas_float].fillna(0)
  data[columnas_float] = data[columnas_float].astype(int)
  data = data.apply(lambda x: x.map(convertir_si_no))


  if size != 0:
    data = data.sample(n=size, random_state=42)
  else:
    pass

  if test== False:

    X = data.iloc[:,:11]
    Y = data.iloc[:,11]
    columnas_cualitativas = X.select_dtypes(include=['object']).columns
    X = pd.get_dummies(X, columns=columnas_cualitativas)
    conversion_dict = {
      'alto': 0,
      'medio-alto': 1,
      'bajo': 2,
      'medio-bajo': 3
    }
    Y = Y.replace(conversion_dict)
    X = X.loc[:,~X.columns.duplicated()].copy()
    return X,Y

  if test == True:

    X = data.iloc[:,:11]
    columnas_cualitativas = X.select_dtypes(include=['object']).columns
    X = pd.get_dummies(X, columns=columnas_cualitativas)
    X = X.loc[:,~X.columns.duplicated()].copy()

    return X, _


In [ ]:
X,Y = preproccesing('train.csv', size=0, test=False)
X.drop('ID', inplace=True, axis=1)

(692500, 12)


In [ ]:
X.columns

Index(['PERIODO', 'ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO',
       'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA',
       'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE',
       'FAMI_EDUCACIONMADRE', 'ESTU_PAGOMATRICULAPROPIO'],
      dtype='object')

In [ ]:
X_train , X_test , y_train , y_test = train_test_split(X.values, Y.values, test_size = 0.2 , random_state = 42)

In [ ]:
print("Train: ",X_train.shape , y_train.shape)
print("Test: ",X_test.shape , y_test.shape)

Train:  (554000, 10) (554000,)
Test:  (138500, 10) (138500,)


In [ ]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)  # Puedes ajustar los hiperparámetros según sea necesario

# Entrenar el modelo
rf_model.fit(X_train, y_train)

# Realizar predicciones
y_pred = rf_model.predict(X_test)

# Calcular la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Precisión: {accuracy:.4f}')

# Obtener un informe de clasificación
print(classification_report(y_test, y_pred))

Precisión: 0.3841
              precision    recall  f1-score   support

           0       0.50      0.54      0.52     35165
           1       0.41      0.45      0.43     34573
           2       0.29      0.26      0.28     34259
           3       0.30      0.28      0.29     34503

    accuracy                           0.38    138500
   macro avg       0.38      0.38      0.38    138500
weighted avg       0.38      0.38      0.38    138500



In [ ]:
X,_ = preproccesing('test.csv', size=0, test=True)

(296786, 12)


In [ ]:
print("Sample size: ",X.shape)

Sample size:  (296786, 11)


In [ ]:
X.columns

Index(['Unnamed: 0', 'ID', 'PERIODO', 'ESTU_PRGM_ACADEMICO',
       'ESTU_PRGM_DEPARTAMENTO', 'ESTU_VALORMATRICULAUNIVERSIDAD',
       'ESTU_HORASSEMANATRABAJA', 'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET',
       'FAMI_EDUCACIONPADRE', 'FAMI_EDUCACIONMADRE'],
      dtype='object')

In [ ]:
IDs = X['ID']
X.drop('ID', inplace=True, axis=1)

In [ ]:
y_pred_test = rf_model.predict(X.values)

In [ ]:
dic = {'ID': IDs.values.T, 'RENDIMIENTO_GLOBAL': y_pred_test.T}

In [ ]:
test_results = pd.DataFrame(data=dic)
test_results

,ID,RENDIMIENTO_GLOBAL
0,550236,1
1,98545,3
2,499179,1
3,782980,3
4,785185,1
...,...,...
296781,496981,1
296782,209415,1
296783,239074,1
296784,963852,1


In [ ]:
conversion_dict = {
      0: 'alto',
      1: 'medio-alto',
      2: 'bajo',
      3: 'medio-bajo'
    }
test_results['RENDIMIENTO_GLOBAL'] = test_results['RENDIMIENTO_GLOBAL'].replace(conversion_dict)

In [ ]:
test_results

,ID,RENDIMIENTO_GLOBAL
0,550236,medio-alto
1,98545,medio-bajo
2,499179,medio-alto
3,782980,medio-bajo
4,785185,medio-alto
...,...,...
296781,496981,medio-alto
296782,209415,medio-alto
296783,239074,medio-alto
296784,963852,medio-alto


In [ ]:
test_results.to_csv('modelo_RF.csv', index=False)